In [1]:
import os, sys, re, pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import tensorflow as tf
import time
from tensorflow import keras

from Data import Pro_data
print(sys.version_info)
for module in tf, mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
tensorflow 2.1.0
matplotlib 3.2.0
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
ORIGIN_DATA_DIR = os.getcwd() + '\\data\\BX-CSV-Dump\\'
FILTERED_DATA_DIR = os.getcwd() + '\\tmp\\'

In [3]:
data = Pro_data.DataLoad()
data.features.head()

,User-ID,Location,ISBN,Title,Author,Year,Publisher,Blurb
0,2918,"[1242, 7251, 5372]",32719,"[18416, 3305, 2794, 2794, 2794, 2794, 2794, 27...",13704,63,2346,"[16743, 70936, 91023, 94647, 30725, 94201, 110..."
1,2918,"[1242, 7251, 5372]",23709,"[5042, 20096, 12301, 3565, 6702, 20408, 4956, ...",6881,71,2064,"[26888, 9440, 6976, 72713, 117446, 20931, 2329..."
2,24974,"[4759, 1444, 5372]",23709,"[5042, 20096, 12301, 3565, 6702, 20408, 4956, ...",6881,71,2064,"[26888, 9440, 6976, 72713, 117446, 20931, 2329..."
3,18233,"[7245, 30, 1600]",23709,"[5042, 20096, 12301, 3565, 6702, 20408, 4956, ...",6881,71,2064,"[26888, 9440, 6976, 72713, 117446, 20931, 2329..."
4,240,"[4723, 6498, 1600]",23709,"[5042, 20096, 12301, 3565, 6702, 20408, 4956, ...",6881,71,2064,"[26888, 9440, 6976, 72713, 117446, 20931, 2329..."


In [4]:
length  = [len(i) for i in data.features['Blurb']]
np.mean(np.array(length))

200.0

In [5]:
data.features.head()

,User-ID,Location,ISBN,Title,Author,Year,Publisher,Blurb
0,2918,"[1242, 7251, 5372]",32719,"[18416, 3305, 2794, 2794, 2794, 2794, 2794, 27...",13704,63,2346,"[16743, 70936, 91023, 94647, 30725, 94201, 110..."
1,2918,"[1242, 7251, 5372]",23709,"[5042, 20096, 12301, 3565, 6702, 20408, 4956, ...",6881,71,2064,"[26888, 9440, 6976, 72713, 117446, 20931, 2329..."
2,24974,"[4759, 1444, 5372]",23709,"[5042, 20096, 12301, 3565, 6702, 20408, 4956, ...",6881,71,2064,"[26888, 9440, 6976, 72713, 117446, 20931, 2329..."
3,18233,"[7245, 30, 1600]",23709,"[5042, 20096, 12301, 3565, 6702, 20408, 4956, ...",6881,71,2064,"[26888, 9440, 6976, 72713, 117446, 20931, 2329..."
4,240,"[4723, 6498, 1600]",23709,"[5042, 20096, 12301, 3565, 6702, 20408, 4956, ...",6881,71,2064,"[26888, 9440, 6976, 72713, 117446, 20931, 2329..."


In [6]:
#User-ID number,max_index
all_user_id_number = len(set(data.features['User-ID']))
print('all_user_id_number=',all_user_id_number)

#Location number index
location_length = len(data.features['Location'][0])
all_location_words_number = max([j for i in data.features['Location'] for j in i])
print('location_length=%d,all_location_words_number=%d '% (location_length,all_location_words_number) )

#ISBN numner
all_isbn_words_number = len(set(data.features['ISBN']))
print('all_isbn_words_number ',all_isbn_words_number)

#Title
title_length = len(data.features['Title'][0])
all_title_words_number = max([j for i in data.features['Title'] for j in i])
print('title_length=%d,all_title_words_number=%d'% (title_length,all_title_words_number))

#Author
all_author_words_number = len(set(data.features['Author']))
print('all_author_words_number ',all_author_words_number)

#Year
all_year_words_number = len(set(data.features['Year']))
print('all_year_words_number ',all_year_words_number)

# Publisher
all_publisher_words_number = len(set(data.features['Publisher']))
print('all_publisher_words_number ',all_publisher_words_number)

# Blurb
blurb_length = len(data.features['Blurb'][0])
all_blurb_words_number = max([j for i in data.features['Blurb'] for j in i])
print('blurb_length=%d,all_blurb_words_number=%d'% (blurb_length,all_blurb_words_number))

all_user_id_number= 28836
location_length=3,all_location_words_number=7573 
all_isbn_words_number  38036
title_length=15,all_title_words_number=23730
all_author_words_number  15196
all_year_words_number  81
all_publisher_words_number  2909
blurb_length=200,all_blurb_words_number=127034


In [7]:
"""
超参数
"""
# Number of Epochs
num_epochs = 5
# Batch_size
batch_size = 256
# dropout_keep
dropout_keep = 0.5
# Learning_rate
learning_rate = 0.001
# embedding_dim
embedding_dim = 16
# 文本卷积滑动的单词个数
window_sizes = {2,3,4,5}
# filter num
filter_num = 8
# dense dim
dense_dim = 32

In [8]:
def get_inputs():
#     用户特征输入占位符
    # 用户特征输入
    user_id = keras.layers.Input(shape=(1,), dtype='int32', name='user_id_input')
    user_location = keras.layers.Input(shape=(3,), dtype='int32', name='user_location_input')
    
    # 书籍特征输入
    book_isbn = keras.layers.Input(shape=(1,),  dtype='int32', name='book_isbn_input')
    book_author = keras.layers.Input(shape=(1,),  dtype='int32', name='book_author_input')
    book_year = keras.layers.Input(shape=(1,),  dtype='int32', name='book_year_input')
    book_publisher = keras.layers.Input(shape=(1,),  dtype='int32', name='book_publisher_input')  
    book_title = keras.layers.Input(shape=(15, ), dtype='int32', name='book_title_input')
    book_blurb = keras.layers.Input(shape=(200, ), dtype='int32', name='book_blurb_input')
    return user_id, user_location, book_isbn, book_author, book_year, book_publisher, book_title, book_blurb

In [9]:
"""
构建User神经网络
"""
def get_user_embedding(user_id,user_location):
    uid_embed_layer = tf.keras.layers.Embedding(all_user_id_number,embedding_dim,input_length=1,name='uid_embed_layer')(user_id)
    location_embed_layer = tf.keras.layers.Embedding(all_location_words_number,embedding_dim,input_length=location_length,name='location_embed_layer')(user_location)
    return uid_embed_layer, location_embed_layer

In [10]:
def get_user_feature_layer(uid_embed_layer,location_embed_layer):
#     第一层全连接
    uid_fc_layer = tf.keras.layers.Dense(dense_dim,name='uid_fc_layer',activation='relu')(uid_embed_layer)
    location_fc_layer = tf.keras.layers.Dense(dense_dim,name='location_fc_layer',activation='relu')(location_embed_layer)
#  对location进行Encoder提取特征
    location_gru_layer = tf.keras.layers.GRU(units=dense_dim,dropout=0.5,name='location_gru_layer')(location_fc_layer)
#     [None,32]
    print(location_gru_layer.shape)
    location_gru_expand_layer = tf.expand_dims(location_gru_layer,axis=1)
    
#     第二层全连接
    user_combine_layer = tf.keras.layers.concatenate([uid_fc_layer,location_gru_expand_layer],2)
    user_dense_layer = tf.keras.layers.Dense(200,activation='tanh',name='user_dense_layer')(user_combine_layer)
    user_dense_layer_flat = tf.keras.layers.Reshape([200], name="user_combine_layer_flat")(user_dense_layer)
    return user_dense_layer,user_dense_layer_flat

In [11]:
"""
构建book神经网络
"""
def get_book_embedding(book_isbn, book_author, book_year, book_publisher, book_title, book_blurb):
    book_isbn_embed_layer = tf.keras.layers.Embedding(all_isbn_words_number,embedding_dim,input_length = 1,name='book_isbn_embed_layer')(book_isbn)
    book_author_embed_layer = tf.keras.layers.Embedding(all_author_words_number,embedding_dim,input_length=1,name='book_author_embed_layer')(book_author)
    book_year_embed_layer = tf.keras.layers.Embedding(all_year_words_number,embedding_dim,input_length=1,name='book_year_embed_layer')(book_year)
    book_publisher_embed_layer = tf.keras.layers.Embedding(all_publisher_words_number,embedding_dim,input_length = 1,name='book_publisher_embed_layer')(book_publisher)
    book_title_embed_layer = tf.keras.layers.Embedding(all_title_words_number,embedding_dim,input_length=title_length,name='book_title_embed_layer')(book_title)
    book_blurb_embed_layer = tf.keras.layers.Embedding(all_blurb_words_number,embedding_dim,input_length = blurb_length,name='book_blurb_embed_layer')(book_blurb)
    return book_isbn_embed_layer,book_author_embed_layer,book_year_embed_layer,book_publisher_embed_layer,book_title_embed_layer,book_blurb_embed_layer
    

In [12]:
def get_book_feature_layer(book_isbn_embed_layer,book_author_embed_layer,book_year_embed_layer,book_publisher_embed_layer,book_title_embed_layer,book_blurb_embed_layer):
#     对isbn,author,year,publisher第一层全连接
    book_isbn_dense_layer = tf.keras.layers.Dense(dense_dim,activation='relu',name='book_isbn_dense_layer')(book_isbn_embed_layer)
    book_author_dense_layer = tf.keras.layers.Dense(dense_dim,activation='relu',name='book_author_dense_layer')(book_author_embed_layer)
    book_year_dense_layer = tf.keras.layers.Dense(dense_dim,activation='relu',name='book_year_dense_layer')(book_year_embed_layer)
    book_publisher_dense_layer = tf.keras.layers.Dense(dense_dim,activation='relu',name='book_publisher_dense_layer')(book_publisher_embed_layer)
    book_title_embed_layer_expand = tf.expand_dims(book_title_embed_layer,axis=-1)
#     对title进行文本卷积
#     book_title_embed_layer_expand:[None,15,16,1]
#     对文本嵌入层使用不同的卷积核做卷积核最大池化
    pool_layer_list = []
    for window_size in window_sizes:
        title_conv_layer = tf.keras.layers.Conv2D(filters = filter_num,kernel_size = (window_size,embedding_dim),strides=1,activation='relu')(book_title_embed_layer_expand)
        title_maxpool_layer = tf.keras.layers.MaxPooling2D(pool_size=(title_length-window_size+1,1),strides=1)(title_conv_layer)
        pool_layer_list.append(title_maxpool_layer)
    pool_layer_layer = tf.keras.layers.concatenate(pool_layer_list,axis=-1,name='title_pool_layer')
    max_num = len(window_sizes)*filter_num
    pool_layer_flat = tf.keras.layers.Reshape([1,max_num],name='pool_layer_flat')(pool_layer_layer)
    dropout_layer = tf.keras.layers.Dropout(dropout_keep, name = "dropout_layer")(pool_layer_flat)

    # 对简介进行Encoder特征提取
    book_blurb_dense_layer = tf.keras.layers.Dense(dense_dim,activation='relu',name='book_blurb_dense_layer')(book_blurb_embed_layer)
    book_blurb_gru_layer = tf.keras.layers.GRU(units=dense_dim,dropout=0.5,name='book_blurb_gru_layer')(book_blurb_dense_layer)
    print('book_blurb_gru_layer=',book_blurb_gru_layer.shape)
    book_blurb_gru_expand_layer = tf.expand_dims(book_blurb_gru_layer,axis=1)
    book_combine_layer = tf.keras.layers.concatenate([book_isbn_dense_layer,book_author_dense_layer,book_year_dense_layer,book_publisher_dense_layer,dropout_layer,book_blurb_gru_expand_layer],axis=-1)
    book_dense_layer = tf.keras.layers.Dense(200, activation='tanh')(book_combine_layer)
    book_dense_layer_flat = tf.keras.layers.Reshape([200], name="book_dense_layer_flat")(book_dense_layer)
    return book_dense_layer,book_dense_layer_flat

In [13]:
def get_rating(user_feature, book_feature):
    multiply_layer = keras.layers.Lambda(lambda layer: tf.reduce_sum(layer[0]*layer[1],axis=1,keepdims=True), name = 'user_book_feature')(user_feature, book_feature)
    print(multiply_layer.shape)
    return multiply_layer

In [14]:
a = tf.constant([[2,1,1],[1,3,4]])
b = tf.constant([[0,1,1],[1,0,4]])
c = inference = tf.keras.layers.Lambda(lambda layer:tf.reduce_sum(layer[0] * layer[1], axis=1,keepdims=True))((a,b))
c.shape

TensorShape([2, 1])

In [15]:
features = data.features.values

In [16]:
features[:2]

array([[2918, list([1242, 7251, 5372]), 32719,
        list([18416, 3305, 2794, 2794, 2794, 2794, 2794, 2794, 2794, 2794, 2794, 2794, 2794, 2794, 2794]),
        13704, 63, 2346,
        list([16743, 70936, 91023, 94647, 30725, 94201, 110675, 90181, 124050, 43353, 17884, 63098, 25481, 55269, 5885, 41278, 4336, 123778, 68058, 98997, 76124, 102306, 1763, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 3630, 363

In [17]:
targets=data.labels.values

In [18]:
# get_Batch
def get_batch(Xs,ys,batchsize):
    for start in range(0,len(Xx,batchsize)):
        end = min(start+batchsize,len(Xs))
        yield xS[start:end],ys[start:end]

# 构建计算图
将用户特征和书籍特征作为输入，经过一个全连接，输出预测

In [19]:
# test
a = np.array([[1, 2, 4, ([1, 2, 5])], 
             [3, 2, 6, ([6, 5, 1])],
             [6, 9, 4, ([3, 7, 5])]])
a.take(3,1)[0]

[1, 2, 5]

In [20]:
class mv_network(object):
    def __init__(self, batch_size=256):
        self.batch_size = batch_size
        self.best_loss = 9999
        self.losses = {'train': [], 'test': []}

        # 获取输入占位符
        user_id, user_location, book_isbn, book_author, book_year, book_publisher, book_title, book_blurb = get_inputs()
        # 获取User的2个嵌入向量
        uid_embed_layer, location_embed_layer = get_user_embedding(user_id,user_location)
        # 得到用户特征
        user_dense_layer,user_dense_layer_flat =get_user_feature_layer(uid_embed_layer,location_embed_layer)
        # 获取书籍的嵌入向量
        book_isbn_embed_layer,book_author_embed_layer,book_year_embed_layer,book_publisher_embed_layer,book_title_embed_layer,book_blurb_embed_layer=get_book_embedding(book_isbn, book_author, book_year, book_publisher, book_title, book_blurb)
        # 获取书籍特征
        book_dense_layer,book_dense_layer_flat=get_book_feature_layer(book_isbn_embed_layer,book_author_embed_layer,book_year_embed_layer,book_publisher_embed_layer,book_title_embed_layer,book_blurb_embed_layer)
        
        # 计算出评分
        # 将用户特征和电影特征做矩阵乘法得到一个预测评分的方案
        print("user_dense_layer_flat=",user_dense_layer_flat.shape)
        print("book_dense_layer_flat=",book_dense_layer_flat.shape)
        inference = get_rating(user_dense_layer_flat, book_dense_layer_flat)

        # 将用户特征和电影特征作为输入，经过全连接，输出一个值的方案
        # inference_layer = tf.keras.layers.concatenate([user_combine_layer_flat, movie_combine_layer_flat],1)  # (?, 400)
        # # 你可以使用下面这个全连接层，试试效果
        # inference_dense = tf.keras.layers.Dense(64, kernel_regularizer=tf.nn.l2_loss, activation='relu')(inference_layer)
        # inference = tf.keras.layers.Dense(1, name="inference")(inference_layer)  # inference_dense

        self.model = tf.keras.Model(
            inputs=[user_id, user_location, book_isbn, book_title, book_author, book_year, book_publisher, book_blurb],
            outputs=[inference])

        self.model.summary()
model = mv_network()

(None, 32)
book_blurb_gru_layer= (None, 32)
user_dense_layer_flat= (None, 200)
book_dense_layer_flat= (None, 200)


ValueError: Invalid reduction dimension 1 for input with 1 dimensions. for 'user_book_feature/Sum' (op: 'Sum') with input shapes: [200], [] and with computed input tensors: input[1] = <1>.